# AutoBuild
By: [Linxin Song](https://linxins97.github.io/), [Jieyu Zhang](https://jieyuz2.github.io/)
Reference: [Agent AutoBuild](https://microsoft.github.io/autogen/blog/2023/11/26/Agent-AutoBuild/)

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we introduce a new class, `AgentBuilder`, to help user build an automatic task solving process powered by multi-agent system. Specifically, in `build()`, we prompt a LLM to create multiple participant agent and initialize a group chat, and specify whether this task need programming to solve. AgentBuilder also support open-source LLMs by [vLLM](https://docs.vllm.ai/en/latest/index.html) and [Fastchat](https://github.com/lm-sys/FastChat). Check the supported model list [here](https://docs.vllm.ai/en/latest/models/supported_models.html).

## Requirement

AutoBuild need the latest version of AutoGen.
You can install AutoGen by the following command:

In [ ]:
!pip install pyautogen

## Step 1: prepare configuration and some useful functions
Prepare a `config_path` for assistant agent to limit the choice of LLM you want to use in this task. This config can be a path of json file or a name of environment variable. A `default_llm_config` is also required for initialize the specific config of LLMs like seed, temperature, etc...

In [1]:
import autogen
from autogen.agentchat.contrib.agent_builder import AgentBuilder

config_path = 'OAI_CONFIG_LIST'  # modify path
default_llm_config = {
    'temperature': 0
}

def start_task(execution_task: str, agent_list: list, llm_config: dict):
    config_list = autogen.config_list_from_json(config_path, filter_dict={"model": ["gpt-4-1106-preview"]})
    
    group_chat = autogen.GroupChat(agents=agent_list, messages=[], max_round=12)
    manager = autogen.GroupChatManager(
        groupchat=group_chat, llm_config={"config_list": config_list, **llm_config}
    )
    agent_list[0].initiate_chat(manager, message=execution_task)

## Step 2: create a AgentBuilder
Create a `AgentBuilder` with the specified `config_path`. AgentBuilder will use `gpt-4` in default to complete the whole process, you can specify the `builder_model` and `agent_model` to other OpenAI model to match your task. 
You can also specify an open-source LLM supporting by vLLM and FastChat, see blog for more details.

In [2]:
builder = AgentBuilder(config_path=config_path, builder_model='gpt-4-1106-preview', agent_model='gpt-4-1106-preview')

## Step 3: specify a building task

Specify a building task with a general description. Building task will help build manager (a LLM) decide what agents should be built.

In [3]:
building_task = "Find a paper on arxiv by programming, and analyze its application in some domain. For example, find a recent paper about gpt-4 on arxiv and find its potential applications in software."

## Step 4: build group chat agents
Use `build()` to let build manager (the specified `builder_model`) complete the group chat agents generation. If you think coding is necessary in your task, you can use `coding=True` to add a user proxy (an automatic code interpreter) into the agent list, like: 
```python
builder.build(building_task, default_llm_config, coding=True)
```
If `coding` is not specified, AgentBuilder will determine on its own whether the user proxy should be added or not according to the task.

In [4]:
agent_list, agent_configs = builder.build(building_task, default_llm_config)

==> Generating agents...
['Data_scientist', 'Machine_learning_engineer', 'Natural_language_processing_specialist', 'Arxiv_API_developer', 'Application_analyst'] are generated.
==> Generating system message...
Preparing system message for Data_scientist
Preparing system message for Machine_learning_engineer
Preparing system message for Natural_language_processing_specialist
Preparing system message for Arxiv_API_developer
Preparing system message for Application_analyst
==> Generating description...
Preparing description for Data_scientist
Preparing description for Machine_learning_engineer
Preparing description for Natural_language_processing_specialist
Preparing description for Arxiv_API_developer
Preparing description for Application_analyst
==> Creating agents...
Creating agent Data_scientist with backbone gpt-4-1106-preview...
Creating agent Machine_learning_engineer with backbone gpt-4-1106-preview...
Creating agent Natural_language_processing_specialist with backbone gpt-4-1106-p

## Step 5: execute task
Let agents generated in `build()` to complete the task collaboratively in a group chat.

In [5]:
start_task(
    execution_task="Find a recent paper about gpt-4 on arxiv and find its potential applications in software.",
    agent_list=agent_list,
    llm_config=default_llm_config
)

User_console_and_Python_code_interpreter (to chat_manager):

Find a recent paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
Arxiv_API_developer (to chat_manager):

To begin our task, I will outline the steps we need to take:

1. **Search for Papers**: We will use the arXiv API to search for recent papers that mention "GPT-4". We'll need to define the query parameters to include the title, abstract, or keywords related to GPT-4.

2. **Retrieve and Filter Results**: Once we have the search results, we'll filter them by date to ensure we're looking at the most recent papers.

3. **Download and Read Files**: We will select a relevant paper from the results and download it to extract information about potential applications in software development.

4. **Analyze Information**: After reading the content of the paper, we will discuss the applications of GPT-4 in software development as

## Step 6 (Optional): clear all agents and prepare for the next task
You can clear all agents generated in this task by the following code if your task is completed or the next task is largely different from the current task. If the agent's backbone is an open-source LLM, this process will also shut down the endpoint server. If necessary, you can use `recycle_endpoint=False` to retain the previous open-source LLMs' endpoint server.

In [6]:
builder.clear_all_agents(recycle_endpoint=True)

All agents have been cleared.


## Save & load configs

You can save all necessary information of the built group chat agents. Here is a case for those agents generated in the above task:
```json
{
    "building_task": "Find a paper on arxiv by programming, and analysis its application in some domain. For example, find a latest paper about gpt-4 on arxiv and find its potential applications in software.",
    "agent_configs": [
        {
            "name": "Data_scientist",
            "model": "gpt-4-1106-preview",
            "system_message": "As a Data Scientist, you are tasked with automating the retrieval and analysis of academic papers from arXiv. Utilize your Python programming acumen to develop scripts for gathering necessary information such as searching for relevant papers, downloading them, and processing their contents. Apply your analytical and language skills to interpret the data and deduce the applications of the research within specific domains.\n\n1. To compile information, write and implement Python scripts that search and interact with online resources, download and read files, extract content from documents, and perform other information-gathering tasks. Use the printed output as the foundation for your subsequent analysis.\n\n2. Execute tasks programmatically with Python scripts when possible, ensuring results are directly displayed. Approach each task with efficiency and strategic thinking.\n\nProgress through tasks systematically. In instances where a strategy is not provided, outline your plan before executing. Clearly distinguish between tasks handled via code and those utilizing your analytical expertise.\n\nWhen providing code, include only Python scripts meant to be run without user alterations. Users should execute your script as is, without modifications:\n\n```python\n# filename: <filename>\n# Python script\nprint(\"Your output\")\n```\n\nUsers should not perform any actions other than running the scripts you provide. Avoid presenting partial or incomplete scripts that require user adjustments. Refrain from requesting users to copy-paste results; instead, use the 'print' function when suitable to display outputs. Monitor the execution results they share.\n\nIf an error surfaces, supply corrected scripts for a re-run. If the strategy fails to resolve the issue, reassess your assumptions, gather additional details as needed, and explore alternative approaches.\n\nUpon successful completion of a task and verification of the results, confirm the achievement of the stated objective. Ensuring accuracy and validity of the findings is paramount. Evidence supporting your conclusions should be provided when feasible.\n\nUpon satisfying the user's needs and ensuring all tasks are finalized, conclude your assistance with \"TERMINATE\"."
        },
        {
            "name": "Research_analyst",
            "model": "gpt-4-1106-preview",
            "system_message": "As a Research Analyst, you are expected to be a proficient AI assistant possessing a strong grasp of programming, specifically in Python, and robust analytical capabilities. Your primary responsibilities will include:\n\n1. Conducting comprehensive searches and retrieving information autonomously through Python scripts, such as querying databases, accessing web services (like arXiv), downloading and reading files, and retrieving system information.\n2. Analyzing the content of the retrieved documents, particularly academic papers, and extracting insights regarding their application in specific domains, such as the potential uses of GPT-4 in software development.\n3. Presenting your findings in a clear, detailed manner, explaining the implications of the research and its relevance to the assigned task.\n4. Employing your programming skills to automate tasks where possible, ensuring the output is delivered through Python code with clear, executable instructions. Your code will be designed for the user to execute without amendment or additional input.\n5. Verifying the results of information gathering and analysis to ensure accuracy and completeness, providing evidence to support your conclusions when available.\n6. Communicating the completion of each task and confirming that the user's needs have been satisfied through a clear and conclusive statement, followed by the word \"TERMINATE\" to signal the end of the interaction."
        },
        {
            "name": "Software_developer",
            "model": "gpt-4-1106-preview",
            "system_message": "As a dedicated AI assistant for a software developer, your role involves employing your Python programming prowess and proficiency in natural language processing to facilitate the discovery and analysis of scholarly articles on arXiv. Your tasks include crafting Python scripts to automatically search, retrieve, and present information regarding the latest research, with a focus on applicable advancements in technology such as GPT-4 and its potential impact on the domain of software development.\n\n1. Utilize Python to programmatically seek out and extract pertinent data, for example, navigating or probing the web, downloading/ingesting documents, or showcasing content from web pages or files. When enough information has been accumulated to proceed, you will then analyze and interpret the findings.\n\n2. When there's a need to perform an operation programmatically, your Python code should accomplish the task and manifest the outcome. Progress through the task incrementally and systematically.\n\nProvide a clear plan outlining each stage of the task, specifying which components will be executed through Python coding and which through your linguistic capabilities. When proposing Python code, remember to:\n\n- Label the script type within the code block\n- Avoid suggesting code that the user would need to alter\n- Refrain from including more than one code block in your response\n- Circumvent requesting the user to manually transcribe any results; utilize 'print' statements where applicable\n- Examine the user's reported execution outcomes\n\nIf an error arises, your responsibility is to rectify the issue and submit the corrected script. Should an error remain unresolvable, or if the task remains incomplete post successful code execution, re-evaluate the scenario, gather any further required information, and formulate an alternative approach.\n\nUpon confirming that the task has been satisfactorily accomplished and the user's requirements have been met, indicate closure of the procedure with a concluding statement."
        }
    ],
    "coding": true,
    "default_llm_config": {
        "temperature": 0
    }
}
```
These information will be saved in JSON format. You can provide a specific filename, otherwise, AgentBuilder will save config to the current path with a generated filename 'save_config_TASK_MD5.json'.

In [7]:
saved_path = builder.save()

Building config saved to ./save_config_2a87f83309bd8b5ebb0423f5570385f5.json


After that, you can load the saved config and skip the building process. AgentBuilder will create agents with those information without prompting the builder manager.

In [8]:
new_builder = AgentBuilder(config_path=config_path)
agent_list, agent_configs = new_builder.load(saved_path)  # load previous agent configs
start_task(
    execution_task="Find a recent paper about Llava on arxiv and find its potential applications in computer vision.",
    agent_list=agent_list,
    llm_config=default_llm_config
)
new_builder.clear_all_agents()

Loading config from ./save_config_2a87f83309bd8b5ebb0423f5570385f5.json
==> Creating agents...
Creating agent Data_scientist with backbone gpt-4-1106-preview...
Creating agent Machine_learning_engineer with backbone gpt-4-1106-preview...
Creating agent Natural_language_processing_specialist with backbone gpt-4-1106-preview...
Creating agent Arxiv_API_developer with backbone gpt-4-1106-preview...
Creating agent Application_analyst with backbone gpt-4-1106-preview...
Adding user console proxy...
User_console_and_Python_code_interpreter (to chat_manager):
Find a recent paper about Llava on arxiv and find its potential applications in computer vision.

--------------------------------------------------------------------------------
Arxiv_API_developer (to chat_manager):

To begin, I will craft a Python script that utilizes the arXiv API to search for recent papers related to "Llava" (assuming "Llava" is a keyword relevant to the field of computer vision; if it's a typo and you meant "Lava"

## Use OpenAI Assistant

[The Assistants API](https://platform.openai.com/docs/assistants/overview) allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries.
AutoBuild also support assistant api by adding `use_oai_assistant=True` to `build()`.

In [4]:
new_builder = AgentBuilder(config_path=config_path, builder_model='gpt-4-1106-preview', agent_model='gpt-4-1106-preview')
agent_list, agent_configs = new_builder.build(building_task, default_llm_config, use_oai_assistant=True)  # Transfer to OpenAI assistant API.
start_task(
    execution_task="Find a recent paper about explainable AI on arxiv and find its potential applications in medical.",
    agent_list=agent_list,
    llm_config=default_llm_config
)
new_builder.clear_all_agents()

==> Generating agents...
['Python_Developer', 'AI_Research_Scientist', 'Application_Analyst', 'Technical_Reviewer'] are generated.
==> Generating system message...
Preparing system message for Python_Developer
Preparing system message for AI_Research_Scientist
Preparing system message for Application_Analyst
Preparing system message for Technical_Reviewer
==> Generating description...
Preparing description for Python_Developer
Preparing description for AI_Research_Scientist
Preparing description for Application_Analyst
Preparing description for Technical_Reviewer
==> Creating agents...
Creating agent Python_Developer with backbone gpt-4-1106-preview...


assistant Python_Developer does not exist, creating a new assistant


Creating agent AI_Research_Scientist with backbone gpt-4-1106-preview...


assistant AI_Research_Scientist does not exist, creating a new assistant


Creating agent Application_Analyst with backbone gpt-4-1106-preview...


assistant Application_Analyst does not exist, creating a new assistant


Creating agent Technical_Reviewer with backbone gpt-4-1106-preview...


assistant Technical_Reviewer does not exist, creating a new assistant


Adding user console proxy...
User_console_and_Python_code_interpreter (to chat_manager):
Find a recent paper about explainable AI on arxiv and find its potential applications in medical.

--------------------------------------------------------------------------------
Python_Developer (to chat_manager):

To accomplish the task of finding recent papers about explainable AI on arXiv and identifying their potential applications in the medical field, we will perform the following steps:

1. Use the arXiv API to search for recent papers on explainable AI.
2. Parse the search results to find a relevant paper.
3. Extract potential applications in the medical field from the abstract or conclusion if available.

Below is the Python script that uses the arXiv API to perform these steps:

```python
import urllib
import urllib.request
import feedparser

# Define the query parameters for the arXiv API
query_params = {
    'search_query': 'all:explainable AI',
    'sortBy': 'submittedDate',
    'sor